# Load modules

In [ ]:
# load modules
import os
import re
import shutil
import blpapi
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from xbbg import blp

sessionOptions = blpapi.SessionOptions()
sessionOptions.setApplicationIdentityKey("52e02b77-2aa7-4412-b718-804f115a0546")
session = blpapi.Session(sessionOptions)
kwargs = {'sess':session}

# Import BoJ holding data

# load table
boj_df = pyarrow.parquet.read_table('./boj_df.parquet').to_pandas()
# boj_df = pd.read_parquet('./boj_df.parquet')
code_table = pd.read_excel('./mei/mei_tbl.xlsx')

mei_files = os.listdir('./mei/in')
for mei_file in mei_files:
    temp_df = pd.read_excel(f'./mei/in/{mei_file}', skiprows=14, usecols=[2,3,4])
    temp_df.columns = ['Type','Series','Amount']
    temp_df = temp_df[~temp_df['Amount'].isna()]
    temp_df['Type'] = temp_df['Type'].ffill()
    temp_df = temp_df.merge(code_table, on=['Type'])
    temp_df['Code'] = [f'{x}{y}' for x,y in zip(temp_df['Category'],temp_df['Series'])]
    temp_df['SortCode'] = [f'{x}{str(y).zfill(3)}' for x,y in zip(temp_df['Category'],temp_df['Series'])]
    temp_df['Date'] = datetime(2000+int(mei_file[3:5]),int(mei_file[5:7]),int(mei_file[7:9]))

    series_meta = blp.bdp(
    tickers=[f'{x} Govt' for x in temp_df['Code']],
    flds=['Maturity','Security_Pricing_Date'],
    **kwargs
    )
    
    series_meta.reset_index(inplace=True)
    series_meta.columns = ['Code','Maturity','Security_Pricing_Date']
    series_meta['Code'] = [x.replace(' Govt','') for x in series_meta['Code']]

    end_date = datetime(2000+int(mei_file[3:5]),int(mei_file[5:7]),int(mei_file[7:9]))
    start_date = end_date + timedelta(days=-10)
    series_ylds = blp.bdh(
    tickers=[f'{code} Govt' for code in temp_df['Code']],
    flds = ["INDEX_YIELD_TO_MATURITY"],
    start_date = start_date,
    end_date = end_date,
    **kwargs
    )    
    series_ylds = series_ylds.mean(axis=0).reset_index()
    series_ylds.columns = ['Code','Ticker','avgYield']
    series_ylds = series_ylds[['Code','avgYield']]
    series_ylds['Code'] = [x.replace(' Govt','') for x in series_ylds['Code']]

    temp_df = temp_df.merge(series_meta, on=['Code'])
    temp_df = temp_df.merge(series_ylds, on=['Code'])
    temp_df['Maturity'] = pd.to_datetime(temp_df['Maturity'])
    temp_df['Security_Pricing_Date'] = pd.to_datetime(temp_df['Security_Pricing_Date'])
    temp_df['Series'] = pd.to_numeric(temp_df['Series'])

    temp_df = temp_df[boj_df.columns]

    # なんかconcatが上手くいかないのでnp.concatenateを使う
    # boj_df = pd.concat([boj_df, temp_df], axis=0, ignore_index=True)
    boj_df = pd.DataFrame(
        data=np.concatenate((boj_df.values, temp_df.values), axis=0),
        columns=boj_df.columns
    )

    shutil.move(f'./mei/in/{mei_file}',f'./mei/{start_date.year}/{mei_file}')

boj_df.to_parquet('./boj_df.parquet', compression='zstd')
# boj_df.to_csv('./boj_df.csv')

# Data handling

In [ ]:
# # create all_mei.csv
# # this csv file gathers all JGB holdings into a long-format dataset, with additional calculation
# #   Amount: holdings on that date
# #   Prev_Amount: holdings on the previous date
# #   D_Amount: increase/decrease in holdings from the previous date (difference of above two)
# #   SortCode: JGB code used for sorting (JGB type + series # with zero-padding)
# #   Maturity: maturity date for each issue
# #   Duration: remaining duration for each issue on that date in years (comparing with maturity, by difference of years and months/12)
# #   avgYield: average yield from previous date (to be considered as the average yield purchased by the BoJ)
# #   stockYield: the average yield of the holding for each issue, calculated as the weighted-mean of avgYield with the weight of D_Amount

# # JGB type to gather (remove JBI and JF for the simplicity)
# sheet_names = ['JN','JS','JSX','JB','JBX','JL','JX','JU','JBI']

# # gather all date
# all_mei = pd.DataFrame(columns=['Code','Date','Amount'])
# for sheet_name in tqdm(sheet_names):
#     temp = pd.read_excel('./JGBbyBoJ.xlsx', sheet_name=sheet_name)
#     temp = temp[temp['Maturity']!='#N/A Invalid Security']
#     temp2 = temp.drop(['Category','Series','Maturity'], axis=1).set_index('Code').stack().reset_index()
#     temp2.columns = ['Code','Date','Amount']
#     temp2 = temp2[temp2['Amount']>0]
#     all_mei = pd.concat([all_mei, temp2], axis=0)

# all_mei = pd.read_excel('./JGBbyBoJ.xlsx', sheet_name='mei')
# all_mei.sort_values(['Code','Date'], inplace=True)

# calculate columns
all_mei = boj_df.copy()
all_mei['Amount'] = all_mei['Amount'].astype(float)
all_mei['avgYield'] = all_mei['avgYield'].astype(float)
all_mei.sort_values(['Code','Date'], inplace=True)
all_mei['Prev_Amount'] = all_mei.groupby(['Code'])['Amount'].shift(1)
all_mei['D_Amount'] = all_mei['Amount'] - all_mei['Prev_Amount']
all_mei.drop(['Prev_Amount'], axis=1, inplace=True)
all_mei['D_Amount'] = all_mei['D_Amount'].fillna(all_mei['Amount'])

# # adding issueList (Maturity, SortCode, etc.)
# issueList = pd.read_excel('./JGBbyBoJ.xlsx', sheet_name='issueList')
# all_mei = issueList.merge(all_mei, on='Code', how='left')
# all_mei = all_mei[all_mei['Amount']>0]

## Used the following lines to create initial dataset (No more needed)
# avgYield = pd.read_excel('./avgYieldbyIssue.xlsx', sheet_name='avgYield')
# avgYield = avgYield.set_index('Date').stack().reset_index()
# avgYield.columns = ['Date','Code','avgYield']
# all_mei = all_mei.merge(avgYield, on=['Code','Date'], how='left')

# # adding average yield
# avgYield = pd.read_excel('./avgYield.xlsx', sheet_name='avgYield')
# all_mei = all_mei.merge(avgYield, on=['Code','Date'], how='left')

# calculate duration and stock yield
all_mei['Duration'] = [(x.year - y.year)+(x.month - y.month)/12 for x,y in zip(all_mei['Maturity'], all_mei['Date'])]
all_mei['D_Amnt*avgYield'] = all_mei['D_Amount']*all_mei['avgYield']
all_mei['stockYield'] = all_mei.groupby('Code')['D_Amnt*avgYield'].cumsum()/all_mei['Amount']
all_mei.drop(['D_Amnt*avgYield'], axis=1, inplace=True)

# sort and save
all_mei = all_mei.sort_values(['SortCode','Date'])
all_mei.to_csv('./all_mei.csv', index=False)
all_mei.to_parquet('./all_mei.parquet', compression='zstd')

In [ ]:
# create pivot table
pivotList = ['Amount','Duration','stockYield']
for col in tqdm(pivotList):
    temp = pd.pivot_table(all_mei[['Date','SortCode',col]], columns='SortCode', index='Date', values=col, aggfunc='sum').reset_index()
    temp.to_csv(f'./mei_{col}.csv', index=False)

In [ ]:
all_mei['stockYield*Amnt'] = all_mei['stockYield']*all_mei['Amount']
all_mei['Duration*Amnt'] = all_mei['Duration']*all_mei['Amount']
all_df = all_mei.groupby('Date')[['Amount','stockYield*Amnt','Duration*Amnt']].sum().reset_index()
all_df['Total_AvgYield'] = all_df['stockYield*Amnt']/all_df['Amount']
all_df['Total_AvgDuration'] = all_df['Duration*Amnt']/all_df['Amount']
all_df.rename(columns={'Amount':'Total_Amnt'}, inplace=True)
all_df = all_df[['Date','Total_Amnt','Total_AvgYield','Total_AvgDuration']]

sheet_names = ['JN','JS','JSX','JB','JBX','JL','JX','JU','JBI']
for category in sheet_names:
    category_mei = all_mei.copy()[all_mei['Category']==category]
    category_mei['stockYield*Amnt'] = category_mei['stockYield']*category_mei['Amount']
    category_mei['Duration*Amnt'] = category_mei['Duration']*category_mei['Amount']
    category_df = category_mei.groupby('Date')[['Amount','stockYield*Amnt','Duration*Amnt']].sum().reset_index()
    category_df[f'{category}_AvgYield'] = category_df['stockYield*Amnt']/category_df['Amount']
    category_df[f'{category}_AvgDuration'] = category_df['Duration*Amnt']/category_df['Amount']
    category_df.rename(columns={'Amount':f'{category}_Amnt'}, inplace=True)
    category_df = category_df[['Date',f'{category}_Amnt',f'{category}_AvgYield',f'{category}_AvgDuration']]

    all_df = all_df.merge(category_df, on='Date', how='left')

In [ ]:
all_df.to_csv('./all_df.csv', index=False)

# Create month-end data

In [ ]:
def getEndOfMonth(df):
    df=df.sort_index()
    return df[(pd.Series(df.index.month.values).diff(-1) != 0).values]

all_mei2 = pd.pivot_table(all_mei, columns='SortCode', index='Date', values='Amount')
all_mei2 = getEndOfMonth(all_mei2)
all_mei2.to_csv('./all_mei2.csv')

# all_mei2['YM'] = [f'{x.year}{str(x.month).zfill(2)}' for x in all_mei2['Date']]
# all_mei2 = all_mei2.groupby(['Code','YM']).tail(1)
# all_mei2 = pd.pivot_table(all_mei2, columns='SortCode', index='Date', values='Amount').dropna(how='all')

In [ ]:
month_list = list(all_mei2.index)
end_month = month_list[-1]
num_add_months = 60
for ix in range(num_add_months):
    new_date = (end_month + relativedelta(months=ix+2)).replace(day=1) - timedelta(days=1)
    month_list.append(new_date)

# month_list.astype('datetime64[D]')
# num_add_months = 60
# new_month_list = np.zeros(len(month_list)+num_add_months)
# new_month_list[:len(month_list)] = month_list
# for ix in range(len(month_list)):
#     new_month_list[ix] = pd.Timestamp(new_month_list[ix])

# for ix in range(num_add_months):
#     new_month_list[len(month_list)+ix] = (pd.Timestamp(new_month_list[len(month_list)+ix-1]) + relativedelta(months=1)).replace(day=1) - timedelta(days=1)

# for ix in range(len(new_month_list)):
#     new_month_list[ix] = pd.Timestamp(new_month_list[ix])

In [ ]:
mei_meta = all_mei[['Code','SortCode','Category','Maturity']].set_index('SortCode').to_dict()
mei_amount_ext = pd.DataFrame(index=month_list, columns=all_mei2.columns)
for sortcode in all_mei2.columns:
    code = mei_meta['Code'][sortcode]
    maturity = mei_meta['Maturity'][sortcode]
    category = mei_meta['Category'][sortcode]
    values = all_mei2[sortcode].values

    new_values = np.zeros(len(month_list))
    new_values[0:len(values)] = values

    if np.isnan(values[-1]):
        new_values[len(values):] = np.nan
    else:
        new_values[len(values):] = [values[-1] if x<maturity else np.nan for x in month_list[len(values):]]

    mei_amount_ext[sortcode] = new_values

mei_long_ext = pd.melt(mei_amount_ext.reset_index(), id_vars=['index'], var_name='SortCode', value_name='Amount').dropna()
mei_long_ext['Category'] = [mei_meta['Category'][x] for x in mei_long_ext['SortCode']]
mei_long_ext['Category'] = [x.replace('X','') if len(x)==3 else x for x in mei_long_ext['Category']]
mei_wide_ext = pd.pivot_table(mei_long_ext, index='index', columns='Category',  values='Amount', aggfunc='sum')

cat_order = ['JN','JS','JB','JL','JX','JU','JBI']
mei_wide_ext = mei_wide_ext[cat_order]/10000

for cat in cat_order:
    mei_wide_ext[f'{cat}_D'] = mei_wide_ext[cat].shift(1) - mei_wide_ext[cat]

mei_wide_ext.to_csv('boj_redemption.csv')